<h1><center>Problem Statement 1 Base Models </center></h1>

### Installing dependencies 

In [ ]:
%pip install wordfreq
%pip install pandas
%pip install sklearn
%pip install nltk

### Importing lib

In [9]:
import pandas as pd 
import pickle
from sklearn.model_selection import train_test_split

## Importing required Lib 

In [12]:
df_vc = pd.read_pickle("Data_VC_Extracted_Scrape_1.pkl")
# Company = 1 means its is a VC
df_vc['Company'] = 1
print("Company = 1 means its is a VC")
df_vc.reset_index(drop=True, inplace=True)



df_non_vc =pd.read_pickle("Data_NON_VC_Extracted_Scrape_1.pkl")
# Company = 0 means NON_VC
df_non_vc['Company'] = 0
print("Company = 0 means NON_VC")
df_non_vc.reset_index(drop=True, inplace=True)

Company = 1 means its is a VC
Company = 0 means NON_VC


## Joining

In [13]:
## Framing two columns as it might get out of shape
df_vc.columns = df_non_vc.columns
frame_combined = pd.concat([df_vc, df_non_vc], ignore_index=True)

## Splitting into Train & Test 

In [21]:

train, test = train_test_split(frame_combined, test_size=0.3)

In [22]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(frame_combined, test_size=0.3)

## SVM BASE MODEL 

In [27]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score
from sklearn.svm import LinearSVC


# Transform each text into a vector of word counts
vectorizer = CountVectorizer(stop_words="english")

In [28]:
training_features = vectorizer.fit_transform(train["tags_in_home"])    
test_features = vectorizer.transform(test["tags_in_home"])

In [29]:
# Training
model = LinearSVC()
model.fit(training_features, train["Company"])
y_pred = model.predict(test_features)

In [30]:
acc = accuracy_score(test["Company"], y_pred)

print("Accuracy on dataset: {:.2f}".format(acc*100))

Accuracy on dataset: 73.33


### Naive Bayes 

In [11]:
import numpy as np
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
from collections import Counter
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfTransformer

In [15]:
frame_combined

,company_name,url,tags_in_home,hrefs,Company
0,Advent International Corp.,www.adventinternational.com,Advent International Search Toggle About us Gl...,"['https://www.adventinternational.com/', 'http...",1
1,Crescent Real Estate Holdings LLC,www.crescent.com,Skip to content Investors Investors Expertise ...,"['https://crescent.com/', 'https://www.icsecur...",1
2,First Reserve Management LP,www.firstreserve.com,INVESTOR LOGIN About Investment Strategy Compe...,"['https://ics.firstreserve.com', 'https://www....",1
3,Freeman Spogli Management Co. LP,www.freemanspogli.com,Home ABOUT Approach History INDUSTRY FOCUS POR...,['https://www.linkedin.com/company/freeman-spo...,1
4,JER Management LLC,www.jer.com,"The J.E Robert Companies (""JER"") have ceased o...",[],1
5,Thomas H. Lee Partners LP,www.thl.com,About THL Our Culture & Values Our Investors a...,['https://services.intralinks.com/logon.html?c...,1
6,Tishman Speyer Properties LP,www.tishmanspeyer.com,Skip to main content Português Français Englis...,"['https://connect.tishmanspeyer.com/', 'https:...",1
7,Vx Capital Partners LLC,www.vxcapital.com,Skip to main content MENU + About Overview Tea...,"['https://vxcapital.com/about/', 'https://vxca...",1
8,"Charterhouse Group, Inc.",www.charterhousegroup.com,This site requires JavaScript and Cookies to b...,[],1
9,"Castle Harlan, Inc.",www.castleharlan.com,Investor Login visit our successor Home About ...,['https://portal.castleharlan.com'],1


In [ ]:
## Base Models 

In [20]:
df = frame_combined[['tags_in_home','Company']].copy()
counter = Counter(df['Company'].tolist())
top_10_varieties = {i[0]: idx for idx, i in enumerate(counter.most_common(10))}
df = df[df['Company'].map(lambda x: x in top_10_varieties)]

description_list = df['tags_in_home'].tolist()
varietal_list = [top_10_varieties[i] for i in df['Company'].tolist()]
varietal_list = np.array(varietal_list)

count_vect = CountVectorizer()
x_train_counts = count_vect.fit_transform(description_list)


tfidf_transformer = TfidfTransformer()
x_train_tfidf = tfidf_transformer.fit_transform(x_train_counts)

train_x, test_x, train_y, test_y = train_test_split(x_train_tfidf, varietal_list, test_size=0.3)

clf = MultinomialNB().fit(train_x, train_y)
y_score = clf.predict(test_x)

n_right = 0
for i in range(len(y_score)):
    if y_score[i] == test_y[i]:
        n_right += 1

print("Accuracy: %.2f%%" % ((n_right/float(len(test_y)) * 100)))

Accuracy: 68.33%
